In [1]:
import numpy as np
import gymnasium as gym

from tensoraerospace.agent.pid import PID
from tensoraerospace.envs.f16.linear_longitudial import LinearLongitudinalF16
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step
from tensoraerospace.benchmark.function import overshoot, settling_time, static_error

In [2]:
dt = 0.01  # Дискретизация
tp = generate_time_period(tn=20, dt=dt) # Временной периуд
tps = convert_tp_to_sec_tp(tp, dt=dt)
number_time_steps = len(tp) # Количество временных шагов
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1]) # Заданный сигнал

pid_config = {'ki': -0.9521120376180967, 'kp': -8.374525879223533, 'kd': 7.281084470573987}

In [3]:
# Инициализация списка для хранения исторических данных
hist = []
dt = 0.01  # Интервал дискретизации времени

# Генерация временного периода с заданным интервалом дискретизации
tp = generate_time_period(tn=40, dt=dt) 

# Конвертация временного периода в секунды
tps = convert_tp_to_sec_tp(tp, dt=dt)

# Вычисление общего количества временных шагов
number_time_steps = len(tp) 

# Создание заданного сигнала с использованием единичного шага
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1])

# Создание среды симуляции, задание временных шагов, начального состояния, заданного сигнала и отслеживаемых состояний
env = gym.make('LinearLongitudinalF16-v0',
               number_time_steps=number_time_steps, 
               initial_state=[[0],[0],[0],[0]],
               reference_signal=reference_signals,
               tracking_states=["alpha"])

# Сброс среды к начальному состоянию
env.reset()

# Инициализация ПИД-регулятора с параметрами из конфигурации
pid = PID(env, kp=pid_config["kp"], ki=pid_config["ki"], kd=pid_config["kd"], dt=dt)

# Начальное состояние системы
xt = np.array([[np.deg2rad(0)], [0]])

# Основной цикл обновления состояния системы и регулятора
for step in range(number_time_steps - 2):
    # Получение текущего заданного значения
    setpoint = reference_signals[0, step]

    # Запись текущего угла в историю
    hist.append(xt[0])

    # Вычисление управляющего сигнала с помощью ПИД-регулятора
    ut  = pid.select_action(setpoint, xt[0])

    # Обновление состояния среды с учетом управляющего сигнала
    xt, reward, terminated, truncated, info = env.step(np.array([ut.item()]))

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the 

In [5]:
system_signal_orig = env.unwrapped.model.get_state('alpha', to_deg=True)[:4000]
control_signal_orig = np.rad2deg(reference_signals[0])[:4000]

In [6]:
# Вызов функции для расчета статической ошибки системы
# Входные данные: исходный управляющий сигнал и исходный сигнал системы
# Функция 'static_error' вычисляет разницу между желаемым и фактическим значением системы в установившемся режиме
static_error(control_signal_orig, system_signal_orig)

0.007530494510000096

In [7]:
# Вызов функции для расчета перерегулирования системы
# Входные данные: исходный управляющий сигнал и исходный сигнал системы
# Функция 'overshoot' определяет максимальное превышение системным сигналом его установившегося значения
overshoot(control_signal_orig, system_signal_orig)

6.11020913086163

In [8]:
# Вызов функции для расчета времени установления системы
# Входные данные: исходный управляющий сигнал и исходный сигнал системы
# Функция 'settling_time' определяет время, необходимое системе для достижения и поддержания значения в пределах заданного диапазона от установившегося значения
# Результат умножается на 0.01 для приведения его к соответствующему временному масштабу
settling_time(control_signal_orig, system_signal_orig) * 0.01

17.61

In [12]:
pid.save()

tensoraerospace.envs.f16.linear_longitudial.LinearLongitudinalF16


/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_init_args to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_init_args` for environment variables or `env.get_wrapper_attr('get_init_args')` that will search the reminding wrappers.
  logger.warn(


In [13]:
agent = PID.from_pretrained("./May03_12-38-38_PID")

In [14]:
agent

In [16]:
# Начальное состояние системы
xt = np.array([[np.deg2rad(0)], [0]])
state, info = env.reset()
# Основной цикл обновления состояния системы и регулятора
for step in range(number_time_steps - 2):
    # Получение текущего заданного значения
    setpoint = reference_signals[0, step]

    # Запись текущего угла в историю
    hist.append(xt[0])

    # Вычисление управляющего сигнала с помощью ПИД-регулятора
    ut  = pid.select_action(setpoint, xt[0])

    # Обновление состояния среды с учетом управляющего сигнала
    xt, reward, terminated, truncated, info = env.step(np.array([ut.item()]))